In [ ]:
from myMongoClient import *
from preparing_finetuning import *
import pandas as pd
%reload_ext autoreload
%autoreload 2

db = "documents-100039"
organisation_id = int(db[-6:])
# mongo_client = MyMongoClient(db)

In [ ]:
labels = mongo_client.get_docs_labels(PREDICTION_DOCUMENT_FOLDER)[0]
dump(labels, "data/labels_results/2024-01-24-labels-for-client.pkl")

## Get a mapping form the file_name and the type of document it is

In [ ]:
label_versions = pd.read_csv("data/labels_results/2024-01-18_16:15:33_df_gold_labels.csv")
output = label_versions[label_versions['label_name'] == 'document type'][['doc_id', 'label_value']]
doc_hash_value = {rows['doc_id'] : rows['label_value'] for idx , rows in output.iterrows()}
file_name_doc_hash = {inst['files'][0]['fileName'] : inst['_id'] for inst in mongo_client.get_all_documents() if inst['files'] and inst['files'][0]['fileName'].startswith(('Pseudo_',"pseudo_"))}
file_nam_value = {key : doc_hash_value[value] for key, value in file_name_doc_hash.items() if value in doc_hash_value}
# file_nam_value, len(file_nam_value)

dump(file_nam_value, "doc_type_from_file_name.pkl")

### Verify that all label name exist in the mapping label_name:label_key

In [ ]:
print_all_missing_mapping()

In [ ]:
def update_mapping_class_label_label_name(label_class, label_name):
    file = load("classifaction_label_to_label_name.pkl")
    file[label_class] = label_name
    dump(file, "classifaction_label_to_label_name.pkl") 

In [ ]:
all_docs_labels = pd.read_csv("data/labels_results/2024-01-12_16:42:23_df_gold_labels.csv")
def  get_label_value(file_name, label_name):
    doc = [doc for doc in all_docs_labels if doc['files'][0]['fileName'] == file_name]
    label = [label for label in doc['labels'] if label['name'] == label_name]
    return label['value'] if label else []
all_prompts = get_all_prompts()
all_prompts['fields_name_value'] = all_prompts.apply(lambda row : 
            {label_name : get_label_value(row['file_name'], label_name)for label_name in row['fields_name_key']}, axis = 1)

In [ ]:
user_llm = {
   "2vIzdtbIlRfELCEIj+Wsxw==" : '7wKQEgxKSZEspILT7n7UqA==',
   "4qvfu2lB0Qe9e6j0Kq+Q0Q==" : "3+h1BtgoAL0PhXWyK1+40g==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "0Dvjg/odKW5noqUL8wrOKw==" : "mHK8QQTTOTp0sx+85PF1qw==",
   "HwklrgHthb7XGEXVDgu6xg==" : "yEpfWSZXTPQZ84K+aj1drQ==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "hcqBfAlCslQ0a1QFu1RcJA==" : "63KSW0tjZ6CrwLEQoCfuUg==",
   "fTXgJLNmmgWMv9xzQjnRpQ==" : "29yradmnIewK1RIRbRSTrg==",
}
def map_doc_id_doc_name() :
    map_doc_id_doc_name = {}
    all_docs = mongo_client.get_all_documents()
    for doc_id in list(all_docs_labels['doc_id'].unique()):
        if doc_id in user_llm :
            doc_id = user_llm[doc_id] 
        map_doc_id_doc_name[doc_id] = [doc['files'][0]['fileName'] for doc in all_docs if doc['_id'] == doc_id and doc['files']][0]
    dump(map_doc_id_doc_name, "map_doc_id_doc_name.pkl")

In [ ]:
all_prompts = get_all_prompts()
all_prompts